In [1]:
#basic imports
import pandas as pd
import numpy as np
from keras.layers import Dense,SimpleRNN,GRU,LSTM,Embedding
from keras.models import Sequential
from tensorflow.keras.utils import to_categorical

In [17]:
#read the data
messages=pd.read_csv(r"E:\uc files\spam.csv",encoding="cp1252")

In [18]:
messages.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [19]:
#keeping necessary columns
messages=messages.loc[:,["v1","v2"]]

In [20]:
#changing columns name
messages=messages.rename(columns={"v1":"label","v2":"message"})

In [21]:
#Converting categorical variable into numerical
messages.label.replace({"spam":1,"ham":0},inplace=True)

In [22]:
#Dividing data into train and test dataset
from sklearn.model_selection import train_test_split

messages_x=messages.iloc[:,1]
messages_y=messages.iloc[:,0]

x_train,x_test,y_train,y_test=train_test_split(messages_x,messages_y,test_size=0.2)

In [23]:
print(x_train.shape,x_test.shape,"----",y_train.shape,y_test.shape)

(4457,) (1115,) ---- (4457,) (1115,)


In [26]:
#like one hot encoding
y_train=to_categorical(y_train)

In [28]:
 y_train

array([[0., 1.],
       [1., 0.],
       [0., 1.],
       ...,
       [0., 1.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [29]:
 y_train.shape

(4457, 2)

In [30]:
max_num_words=10000    #how many words you want from entire corpus
seq_len=50             #how many words from each document
embedding_size=100     #vector length of each word

In [62]:
from keras.preprocessing.text import Tokenizer
#from keras.preprocessing.sequence import pad_sequences
from keras.utils import pad_sequences

In [32]:
tokenizer=Tokenizer(num_words=max_num_words)
tokenizer.fit_on_texts(messages.message)

In [33]:
#assign a token(unique number) to each word
x_train=tokenizer.texts_to_sequences(x_train)
x_test=tokenizer.texts_to_sequences(x_test)

In [34]:
x_train

[[6,
  28,
  178,
  517,
  15,
  530,
  514,
  595,
  186,
  112,
  1168,
  21,
  73,
  5,
  338,
  295,
  2,
  864,
  2824,
  141,
  3843,
  188,
  2825,
  2826,
  1162,
  2827],
 [83,
  415,
  110,
  3103,
  46,
  367,
  32,
  7398,
  24,
  176,
  32,
  51,
  11,
  1814,
  36,
  74,
  2966,
  4214,
  145,
  4,
  582],
 [106, 16, 6309, 1054, 72, 67, 9, 125, 192, 148, 258, 12, 3],
 [180, 137, 24, 14, 33, 1288, 115, 8226, 2, 13, 65],
 [265, 31, 755, 2, 563, 2, 956, 287, 1, 54, 4, 345, 15, 4686],
 [46, 10, 372, 384, 134],
 [38, 1649, 491, 5645],
 [3,
  22,
  300,
  1756,
  76,
  91,
  707,
  227,
  76,
  222,
  3,
  54,
  2,
  169,
  52,
  129,
  14,
  9,
  16,
  48,
  4,
  675,
  310,
  7254,
  4158],
 [69,
  780,
  7037,
  52,
  69,
  4138,
  10,
  69,
  176,
  32,
  232,
  7,
  7038,
  1470,
  10,
  7,
  57,
  1,
  294,
  394,
  4,
  338,
  48,
  130],
 [138, 20, 160, 42, 6768, 4085, 253],
 [122, 122, 86, 3, 151],
 [372, 3045, 588, 144, 492, 6],
 [1076,
  61,
  6554,
  2903,
  4019,
 

In [41]:
x_train[2]

[106, 16, 6309, 1054, 72, 67, 9, 125, 192, 148, 258, 12, 3]

In [43]:
#padding
x_train=pad_sequences(x_train,maxlen=seq_len)
x_test=pad_sequences(x_test,maxlen=seq_len)

In [44]:
x_train[2]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,  106,   16, 6309, 1054,   72,   67,    9,
        125,  192,  148,  258,   12,    3])

In [56]:
model=Sequential()                           #initialize the network
model.add(Embedding(input_dim=max_num_words, #embedding takes input in sequence and gives output in embeddings
                   input_length=seq_len,
                   output_dim=embedding_size))

In [57]:
model.add(LSTM(32))                         #using LSTM with 32neuron
model.add(Dense(2,activation="softmax"))    #output 2 because 2 target variable
from tensorflow.keras.optimizers import Adam
adam=Adam(learning_rate=0.04)
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])

In [58]:
#model building
model.fit(x_train,y_train,epochs=5,batch_size=32,validation_split=0.2)

Epoch 1/5
112/112 [==============================] - 11s 60ms/step - loss: 0.3800 - accuracy: 0.8850 - val_loss: 0.1925 - val_accuracy: 0.9563
Epoch 2/5
112/112 [==============================] - 6s 50ms/step - loss: 0.1237 - accuracy: 0.9792 - val_loss: 0.1053 - val_accuracy: 0.9843
Epoch 3/5
112/112 [==============================] - 6s 50ms/step - loss: 0.0611 - accuracy: 0.9947 - val_loss: 0.0721 - val_accuracy: 0.9854
Epoch 4/5
112/112 [==============================] - 6s 51ms/step - loss: 0.0326 - accuracy: 0.9978 - val_loss: 0.0597 - val_accuracy: 0.9854
Epoch 5/5
112/112 [==============================] - 6s 53ms/step - loss: 0.0195 - accuracy: 0.9986 - val_loss: 0.0511 - val_accuracy: 0.9888


In [59]:
#doeing prediction 
pred=model.predict(x_test)
pred

35/35 [==============================] - 1s 8ms/step


array([[0.98659384, 0.01340612],
       [0.9867743 , 0.01322567],
       [0.05138638, 0.9486136 ],
       ...,
       [0.98172736, 0.01827269],
       [0.9942783 , 0.00572168],
       [0.9574738 , 0.04252614]], dtype=float32)

In [60]:
#returning maximum value along with axis
pred=np.argmax(pred,axis=1)
pred

array([0, 0, 1, ..., 0, 0, 0], dtype=int64)

In [61]:
#building confusion matrix
from sklearn.metrics import confusion_matrix

tab=confusion_matrix(y_test,pred)
tab

array([[957,   5],
       [  9, 144]], dtype=int64)